In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
df_foursq= pd.read_csv("datos/api_foursquare.csv", index_col=0)
df_aemet= pd.read_csv("datos/df_aemet_final.csv")
df_municipios= pd.read_csv("datos/municipios.csv", index_col=0)

In [3]:
df_municipios.sample(2)

,acebeda-la,ajalvir,alameda-del-valle,alamo-el,alcala-de-henares,alcobendas,alcorcon,aldea-del-fresno,algete,alpedrete,...,villamanta,villamantilla,villanueva-de-la-canada,villanueva-del-pardillo,villanueva-de-perales,villar-del-olmo,villarejo-de-salvanes,villaviciosa-de-odon,villavieja-del-lozoya,zarzalejo
2,La Acebeda,Ajalvir,Alameda del Valle,El Álamo,Alcalá de Henares,Alcobendas,Alcorcón,Aldea del Fresno,Algete,Alpedrete,...,Villamanta,Villamantilla,Villanueva de la Cañada,Villanueva del Pardillo,Villanueva de Perales,Villar del Olmo,Villarejo de Salvanés,Villaviciosa de Odón,Villavieja del Lozoya,Zarzalejo
0,41.0869583,40.5342302,40.9177178,40.2307296,40.4818396,40.5400082,40.3492998,40.3215212,40.5961652,40.6599005,...,40.2990939,40.3421033,40.4472106,40.4949232,40.3466066,40.3364465,40.1683582,40.3573787,41.0063622,40.5483016


In [4]:
df_foursq["id_municipio_cambiar"]= df_foursq["municipio"]
df_foursq.drop(columns="municipio", inplace=True)

In [5]:
df_foursq.sample(2)

,category,fsq_id,closed_bucket,distance,link,name,address,latitude,longitude,id_municipio_cambiar
521,Park,50a8c246e4b007904b63d47b,LikelyOpen,828.0,/v3/places/50a8c246e4b007904b63d47b,Parque de las aventuras,NaN,40.324132,-3.874688,mostoles
493,Park,5783c72d38faa3a2ed2b67aa,LikelyOpen,1616.0,/v3/places/5783c72d38faa3a2ed2b67aa,Parque Piscis,28231 Las Rozas Comunidad de Madrid,40.487602,-3.872766,majadahonda


In [6]:
df_aemet.sample(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,localizacion_id
1861,2024-08-29 23:00:00,Muy nuboso con lluvia,20,20,32.0,2.0,0,82,Riesgo,0,65,Riesgo,E,14,cobena-id28041
900,2024-08-30 06:00:00,Cubierto,14,14,8.0,0.0,0,100,Riesgo,0,10,Sin riesgo,SE,3,berzosa-del-lozoya-id28020


Separamos localización id 

In [7]:
df_aemet["localizacion_id"]

0       acebeda-la-id28001
1       acebeda-la-id28001
2       acebeda-la-id28001
3       acebeda-la-id28001
4       acebeda-la-id28001
               ...        
7880     zarzalejo-id28183
7881     zarzalejo-id28183
7882     zarzalejo-id28183
7883     zarzalejo-id28183
7884     zarzalejo-id28183
Name: localizacion_id, Length: 7885, dtype: object

In [8]:
df_aemet["id_municipio"]=pd.DataFrame(df_aemet["localizacion_id"].str.split("\.*-id\.*").str[1])
df_aemet["id_localizacion"]= pd.DataFrame(df_aemet["localizacion_id"].str.split("\.*-id\.*").str[0])
df_aemet.drop(columns="localizacion_id", inplace=True)



In [9]:
df_aemet.head(2)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,id_municipio,id_localizacion
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,28001,acebeda-la
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,28001,acebeda-la


Traspongo municipios para que se convierta el municipio en su id

In [10]:
df_municipioss= df_municipios.T
df_municipioss=df_municipioss.reset_index()

Nombro las columnas

In [11]:
lista= ["id_municipio_cambiar","latitude","longitude","nombre_municipio"]
df_municipioss.columns= lista
df_municipioss.head(2)

,id_municipio_cambiar,latitude,longitude,nombre_municipio
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir


Meter id_municipio en tods las tablas

In [ ]:
df_aemet.head(4)

,fecha,cielo,temp._(°c),sen._térmica_(°c),racha_máx._(km/h),precipitación_(mm),nieve_(mm),humedad_relativa_(%),prob._precip.__(%),prob._de_nieve_(%),prob._de_tormenta_(%),avisos,dirección_viento,velocidad_del_viento,id_municipio,id_localizacion
0,2024-08-29 12:00:00,Nubes altas,21,21,25.0,0.0,0,69,95,0,75,Riesgo,S,7,28001,acebeda-la
1,2024-08-29 13:00:00,Nubes altas,24,24,29.0,0.0,0,57,Riesgo,0,75,Riesgo,SE,12,28001,acebeda-la
2,2024-08-29 14:00:00,Muy nuboso,23,23,33.0,0.0,0,59,100,0,80,Riesgo,S,14,28001,acebeda-la
3,2024-08-29 15:00:00,Nubes altas,23,23,35.0,0.0,0,55,Riesgo,0,80,Riesgo,S,17,28001,acebeda-la


In [13]:
df_municipioss.head(4)

,id_municipio_cambiar,latitude,longitude,nombre_municipio
0,acebeda-la,41.0869583,-3.624399,La Acebeda
1,ajalvir,40.5342302,-3.4807818,Ajalvir
2,alameda-del-valle,40.9177178,-3.8438216,Alameda del Valle
3,alamo-el,40.2307296,-3.9905893,El Álamo


Monto las tablas que no tienen foreign keys

In [14]:
def crear_df_id (df, col):
    df_nuevo= pd.DataFrame(df[col])
    df_nuevo[f"id_{col}"]= pd.RangeIndex(start=1,stop = len(df_nuevo)+1,step=1)
    return df_nuevo

In [15]:
lista_cielo=[]
for nombre in df_aemet["cielo"].unique():
    lista_cielo.append(nombre)
lista_cielo   

['Nubes altas',
 'Muy nuboso',
 'Cubierto con tormenta y lluvia escasa',
 'Nuboso',
 'Cielo despejado',
 'Poco nuboso',
 'Intervalos nubosos con lluvia escasa',
 'Cubierto',
 'Cubierto con lluvia escasa',
 'Cubierto con lluvia',
 'Muy nuboso con tormenta y lluvia escasa',
 'Muy nuboso con lluvia',
 'Intervalos nubosos',
 'Muy nuboso con lluvia escasa',
 'Nuboso con tormenta y lluvia escasa',
 'Cubierto con tormenta',
 'Nuboso con lluvia escasa',
 'Intervalos nubosos con lluvia',
 'Bruma',
 'Niebla',
 'Nuboso con lluvia',
 'Muy nuboso con tormenta',
 'Nuboso con tormenta']

In [16]:
df_cielo1= pd.DataFrame(lista_cielo)
df_cielo1= df_cielo1.rename(columns={0 : "cielo"})
df_cielo= crear_df_id(df_cielo1,"cielo")
df_cielo.head()

,cielo,id_cielo
0,Nubes altas,1
1,Muy nuboso,2
2,Cubierto con tormenta y lluvia escasa,3
3,Nuboso,4
4,Cielo despejado,5


In [17]:
lista_avisos=[]
for nombre in df_aemet["avisos"].unique():
    lista_avisos.append(nombre)
lista_avisos   

['Riesgo', 'Sin riesgo']

In [18]:
df_avisos1= pd.DataFrame(lista_avisos)
df_avisos1= df_avisos1.rename(columns={0 : "avisos"})
df_avisos= crear_df_id(df_avisos1,"avisos")
df_avisos.head()

,avisos,id_avisos
0,Riesgo,1
1,Sin riesgo,2


In [19]:
lista_categorias=[]
for nombre in df_foursq["category"].unique():
    lista_categorias.append(nombre)
lista_categorias

['Park',
 'Monument',
 'TV Station',
 'Castle',
 'Film Studio',
 'Palace',
 'Fountain',
 'Bridge',
 'Beach',
 'Print, TV, Radio and Outdoor Advertising Service']

In [20]:
df_categorias1= pd.DataFrame(lista_categorias)
df_categorias1= df_categorias1.rename(columns={0 : "categorias"})
df_categorias= crear_df_id(df_categorias1,"categorias")
df_categorias.head()

,categorias,id_categorias
0,Park,1
1,Monument,2
2,TV Station,3
3,Castle,4
4,Film Studio,5


In [21]:
lista_estados=[]
for nombre in df_foursq["closed_bucket"].unique():
    lista_estados.append(nombre)
lista_estados

['LikelyOpen', 'VeryLikelyOpen', 'Unsure']

In [22]:
df_estados1= pd.DataFrame(lista_estados)
df_estados1= df_estados1.rename(columns={0 : "estados"})
df_estados= crear_df_id(df_estados1,"estados")
df_estados.head()

,estados,id_estados
0,LikelyOpen,1
1,VeryLikelyOpen,2
2,Unsure,3


In [23]:
lista_mun=[]
for nombre in df_municipioss["nombre_municipio"].unique():
    lista_mun.append(nombre)
# lista_mun  

In [24]:
df_mun1= pd.DataFrame(lista_mun)
df_mun1= df_mun1.rename(columns={0 : "municipios"})
df_municipiosdic= crear_df_id(df_mun1,"municipios")
df_municipiosdic.head(2)

,municipios,id_municipios
0,La Acebeda,1
1,Ajalvir,2


Pasamos a las siguientes tablas con menos ids 1ºmunicipio

In [29]:
df= pd.DataFrame()
df["nombre_municipio"]= df_municipioss["nombre_municipio"]
df["latitud"]= df_municipioss["latitude"]
df["longitud"]= df_municipioss["longitude"]
df["municipio"]= df_municipioss["id_municipio_cambiar"]
df_municipios1= df
df_municipios1.head(2)

,nombre_municipio,latitud,longitud,municipio
0,La Acebeda,41.0869583,-3.624399,acebeda-la
1,Ajalvir,40.5342302,-3.4807818,ajalvir


In [30]:

diccionario1= {}
datos = df_municipiosdic.groupby("municipios")["id_municipios"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario1[datos_cielo[i]] = lista_estados[i]

df_municipios1["id_municipio"] = df_municipios1["nombre_municipio"].map(diccionario1)
df_municipios1.drop(columns="municipio", inplace=True) 

In [31]:

df_municipios1

,nombre_municipio,latitud,longitud,id_municipio
0,La Acebeda,41.0869583,-3.624399,1
1,Ajalvir,40.5342302,-3.4807818,2
2,Alameda del Valle,40.9177178,-3.8438216,3
3,El Álamo,40.2307296,-3.9905893,4
4,Alcalá de Henares,40.4818396,-3.3644973,5
...,...,...,...,...
171,Villar del Olmo,40.3364465,-3.2355624,172
172,Villarejo de Salvanés,40.1683582,-3.2738764,173
173,Villaviciosa de Odón,40.3573787,-3.9002334,174
174,Villavieja del Lozoya,41.0063622,-3.6716291,175


Pasamos a las siguientes tablas con menos ids 2ºtiempo

In [ ]:
df= pd.DataFrame()
df["fecha"]= df_aemet["fecha"]
df["temperatura_°c"]= df_aemet["temp._(°c)"]
df["precipitacion_mm"]= df_aemet["precipitación_(mm)"]
df["humedad_%"]= df_aemet["humedad_relativa_(%)"]
df["avisos"]= df_aemet["avisos"]
df["cielo"]= df_aemet["cielo"]
df["municipio"]= df_municipioss["nombre_municipio"]
df_tiempo= df
df_tiempo.head(2)

,fecha,temperatura_°c,precipitacion_mm,humedad_%,avisos,cielo,municipio
0,2024-08-29 12:00:00,21,0.0,69,Riesgo,Nubes altas,La Acebeda
1,2024-08-29 13:00:00,24,0.0,57,Riesgo,Nubes altas,Ajalvir


In [39]:
df_tiempo["id_municipio"] = df_tiempo["municipio"].map(diccionario1)

In [40]:
diccionario= {}
datos = df_cielo.groupby("cielo")["id_cielo"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario[datos_cielo[i]] = lista_estados[i]

df_tiempo["id_cielo"] = df_tiempo["cielo"].map(diccionario)

In [41]:

diccionario= {}
datos = df_avisos.groupby("avisos")["id_avisos"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario[datos_cielo[i]] = lista_estados[i]

df_tiempo["id_avisos"] = df_tiempo["avisos"].map(diccionario)

df_tiempo.drop(columns="cielo", inplace=True)
df_tiempo.drop(columns="avisos", inplace=True) 
df_tiempo.drop(columns="municipio", inplace=True) 

In [42]:
df_tiempo.head(3) 

,fecha,temperatura_°c,precipitacion_mm,humedad_%,id_municipio,id_cielo,id_avisos
0,2024-08-29 12:00:00,21,0.0,69,1.0,1,1
1,2024-08-29 13:00:00,24,0.0,57,2.0,1,1
2,2024-08-29 14:00:00,23,0.0,59,3.0,2,1


Hacemos la última tabla (lugar)

In [32]:
df= pd.DataFrame()
df["nombre"]= df_foursq["name"]
df["distancia"]= df_foursq["distance"]
df["categoria"]= df_foursq["category"]
df["estado"]= df_foursq["closed_bucket"]
df["municipio"]= df_municipioss["nombre_municipio"]
df[]
df_lugar= df
df_lugar.head(2)

,nombre,distancia,categoria,estado,municipio
0,Diverjungla,563.0,Park,LikelyOpen,La Acebeda
1,Parque Infantil,1724.0,Park,VeryLikelyOpen,Ajalvir


In [33]:
# diccionario= {}
# datos = df_lugdic.groupby("nombre_lugares")["id_nombre_lugares"].first()
# datos_cielo = list(datos.index)
# lista_estados = list(datos.values)
# for i in range(0, len(datos_cielo)):
#     diccionario[datos_cielo[i]] = lista_estados[i]
df_lugar["id_tiempo"] = df_lugar["municipio"].map(diccionario1)
df_lugar["id_municipio"] = df_lugar["municipio"].map(diccionario1)

In [34]:

diccionario= {}
datos = df_categorias.groupby("categorias")["id_categorias"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario[datos_cielo[i]] = lista_estados[i]

df_lugar["id_categoria"] = df_lugar["categoria"].map(diccionario)

In [35]:

diccionario= {}
datos = df_estados.groupby("estados")["id_estados"].first()
datos_cielo = list(datos.index)
lista_estados = list(datos.values)
for i in range(0, len(datos_cielo)):
    diccionario[datos_cielo[i]] = lista_estados[i]

df_lugar["id_estados"] = df_lugar["estado"].map(diccionario)

df_lugar.drop(columns="categoria", inplace=True)
df_lugar.drop(columns="estado", inplace=True) 
df_lugar.drop(columns="municipio", inplace=True) 

In [37]:

df_lugar.head(8)

,nombre,distancia,municipio,id_municipio,id_categoria,id_estados
0,Diverjungla,563.0,La Acebeda,1.0,1,1
1,Parque Infantil,1724.0,Ajalvir,2.0,1,2
2,Chiquitin,333.0,Alameda del Valle,3.0,1,1
3,Isla Perejil,118.0,El Álamo,4.0,1,1
4,Parque de Madera,582.0,Alcalá de Henares,5.0,1,1
5,Parque O'Donnell,593.0,Alcobendas,6.0,1,2
6,Parque del Louisi,326.0,Alcorcón,7.0,1,2
7,Parque Infantil,392.0,Aldea del Fresno,8.0,1,2


Comprobamos nulos y tipos de cada DF

In [14]:
df_aemet["fecha"]= pd.to_datetime(df_aemet["fecha"])

In [40]:
df_aemet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7885 entries, 0 to 7884
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   index                  7885 non-null   int64         
 1   fecha                  7885 non-null   datetime64[ns]
 2   cielo                  7885 non-null   object        
 3   temp._(°c)             7885 non-null   int64         
 4   sen._térmica_(°c)      7885 non-null   int64         
 5   racha_máx._(km/h)      7885 non-null   float64       
 6   precipitación_(mm)     7885 non-null   float64       
 7   nieve_(mm)             7885 non-null   int64         
 8   humedad_relativa_(%)   7885 non-null   int64         
 9   prob._precip.__(%)     7885 non-null   object        
 10  prob._de_nieve_(%)     7885 non-null   int64         
 11  prob._de_tormenta_(%)  7885 non-null   int64         
 12  avisos                 7885 non-null   object        
 13  dir

In [42]:
df_foursq["address"].fillna("sin_informacion", inplace= True)

In [43]:
df_foursq.info()

<class 'pandas.core.frame.DataFrame'>
Index: 907 entries, 0 to 906
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   category       907 non-null    object 
 1   fsq_id         907 non-null    object 
 2   closed_bucket  907 non-null    object 
 3   distance       907 non-null    float64
 4   link           907 non-null    object 
 5   name           907 non-null    object 
 6   address        907 non-null    object 
 7   latitude       907 non-null    float64
 8   longitude      907 non-null    float64
 9   id_municipio   907 non-null    object 
dtypes: float64(3), object(7)
memory usage: 77.9+ KB


Traspongo municipios para que se convierta el municipio en su id

Transformo las coordenadas a float

In [33]:
df_municipioss["latitude"]= df_municipioss["latitude"].astype(float,)
df_municipioss["longitude"]= df_municipioss["longitude"].astype(float)

In [34]:
df_municipioss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_municipio      176 non-null    object 
 1   latitude          176 non-null    float64
 2   longitude         176 non-null    float64
 3   nombre_municipio  176 non-null    object 
dtypes: float64(2), object(2)
memory usage: 5.6+ KB


**NUEVO COMIENZO TENIENDO CLARA LA ESTRUCTURA**

EMPEZAMS POR MUNICIPIO

In [45]:
df_municipioss

,id_municipio,latitude,longitude,nombre_municipio
0,acebeda-la,41.086958,-3.624399,La Acebeda
1,ajalvir,40.534230,-3.480782,Ajalvir
2,alameda-del-valle,40.917718,-3.843822,Alameda del Valle
3,alamo-el,40.230730,-3.990589,El Álamo
4,alcala-de-henares,40.481840,-3.364497,Alcalá de Henares
...,...,...,...,...
171,villar-del-olmo,40.336447,-3.235562,Villar del Olmo
172,villarejo-de-salvanes,40.168358,-3.273876,Villarejo de Salvanés
173,villaviciosa-de-odon,40.357379,-3.900233,Villaviciosa de Odón
174,villavieja-del-lozoya,41.006362,-3.671629,Villavieja del Lozoya
